In [ ]:

import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
import pytesseract
import json
import re
import timeit
import os
from skimage.filters import threshold_local

In [ ]:
#Ordered point for 4 points: top left, top right, bottom left, bottom right
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[3] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[2] = pts[np.argmax(diff)]
    return rect

In [ ]:
#OpenCV perspective transform knowing 4 point and image
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, bl, br) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - br[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA),int(heightB))
    dst = np.array([
        [0,0],
        [maxWidth - 1, 0],
        [0, maxHeight - 1],
        [maxWidth - 1, maxHeight - 1]], dtype = "float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped 
def showimage(image):
    cv2.imshow("pic", cv2.resize(image, (800, int(image.shape[1] / image.shape[0] * 800)), interpolation = cv2.INTER_AREA))
    cv2.waitKey(0)

In [ ]:
img = cv2.imread('business_cards/Canon/102.jpg')
orig = img.copy()
ratio = img.shape[0] / 2000.0
img = cv2.resize(img, (2000, int(img.shape[1] / ratio)), interpolation = cv2.INTER_AREA)
gray = cv2.bilateralFilter(img, 25, 20, 50)
v = np.median(gray)
sigma = 0.8
#---- apply automatic Canny edge detection using the computed median----
lower = int(max(0, (1.0 - sigma) * v))
upper = int(min(255, (1.0 + sigma) * v))
edged = cv2.Canny(gray, lower, upper)
#edges = cv2.Canny( gray, 50, 150, apertureSize = 3)
showimage(edged)
lines = cv2.HoughLines( edged, 1, np.pi/180, 200)
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 5)

showimage(img)
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread('business_cards/Canon/102.jpg')
orig = img.copy()
showimage(img)
ratio = img.shape[0] / 2000.0
img = cv2.resize(img, (2000, int(img.shape[1] / ratio)), interpolation = cv2.INTER_AREA)
#gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(img, 25, 20, 50)
v = np.median(gray)
sigma = 0.8
#---- apply automatic Canny edge detection using the computed median----
lower = int(max(0, (1.0 - sigma) * v))
upper = int(min(255, (1.0 + sigma) * v))
edged = cv2.Canny(gray, lower, upper)
#edges = cv2.Canny( gray, 50, 150, apertureSize = 3)
showimage(edged)
minLineLength = 100
maxLineGap = 10
lines = cv2.HoughLinesP(edged, 1, np.pi/180, 100, minLineLength, maxLineGap)
for x1,y1,x2,y2 in lines[0]:
    cv2.line(edged, (x1,y1), (x2,y2), (0,255,0), 2)

showimage(edged)
cv2.destroyAllWindows()

In [ ]:
def get_scanned(image):
    orig = image.copy()
    ratio = image.shape[0] / 2000.0
    image = cv2.resize(image, (2000, int(image.shape[1] / ratio)), interpolation = cv2.INTER_AREA)
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(image, 9, 25, 175)
#     v = np.median(gray)
#     sigma = 0.9
#     #---- apply automatic Canny edge detection using the computed median----
#     lower = int(max(0, (1.0 - sigma) * v))
#     upper = int(min(255, (1.0 + sigma) * v))
#     edged = cv2.Canny(gray, lower, upper)
#     #showimage(edged)
#     edgeds = cv2.morphologyEx(edged, cv2.MORPH_CROSS, (150,150))
    
    edges = cv2.Canny(gray, 50, 150)
    #showimage(edgeds)
    im2, contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image, contours, -1, (0,255,0), 3)
    showimage(image)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
    rects = []
    screenCnt = np.zeros((4,2))
    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.005 * peri, True)
        x, y, w, h = cv2.boundingRect(approx)
        if h >= 150 and w >= 200 and len(approx) == 4:
            screenCnt = approx
            #print(screenCnt)
            cv2.drawContours(image, [screenCnt], -1, (0,255,0), 3)
            break   
    
    showimage(image)
    warped = four_point_transform(image, screenCnt.reshape(4, 2))
    return warped
image_locale = "business_cards/Canon/105.jpg"
image = cv2.imread(image_locale)
scanned = get_scanned(image)
plt.subplot(121),plt.imshow(cv2.cvtColor(scanned, cv2.COLOR_BGR2RGB)),plt.title('Scanned')
cv2.destroyAllWindows()